In [89]:
import numpy as np
import re

In [90]:
def getNodes(graph):
    nodes = []
    for n in graph.keys():
        nodes.append(n)
    for n2 in graph.values():
        if type(n2) == "string":
            nodes.append(n2)
        else:
            nodes += n2
    return np.unique(nodes).tolist()
    

def getEdges(graph, isDirected):
    edges = []
    for item in graph.items():
        for destination in item[1]:
            if (not isDirected):
                string1 = item[0] if item[0] < destination else destination
                string2 = destination if item[0] < destination else item[0]
            else:
                string1 = item[0]
                string2 = destination
            edges.append(string1 + string2)
    return np.unique(edges).tolist()

In [91]:
graph0 = {
    "a": ["b"]
}
#A---B

graph1 = {
    "a": ["b"],
    "b": ["c"]
}
#A----B----C

graph2 = {
    "a": ["b", "c", "d"],
    "b": ["a"],
    "c": ["a"],
    "d": ["a"]
}

#       C          D
#        \        /
#         \      /
#          \    /
#           \  /
# B----------A

graph3 = {
    "a": "b",
    "b": "c",
    "c": "d"
}
# A--B--C--D

graph4 = {
    "a": "b",
    "b": "c",
    "c": "d",
    "d": "e"
}
# A--B--C--D--E

graph5 = {
    "a": ["b", "c", "d", "e"]
}
#       C          D
#        \        /
#         \      /
#          \    /
#           \  /
# B----------A-------------E

In [92]:
class Player:
    def __init__(self, types, shorthand, roundNum):
        self.types = types
        self.shorthand = shorthand
        self.roundNum = roundNum
    def self_introduce(self):
        return f'Player {self.shorthand} of type {self.types} for round {self.roundNum}'

def buildPlayers(nodes, edges, covers):
    # objectives
    objectiveplayer = [Player('objective', 'o', 1)]
    # nodes
    n = len(nodes)
    vertexplayers = [Player('vertex', 'v0', 1)]
    for i in range(1, n+1):
        vertexplayers.append(Player('vertex', f'v{i}', 1))
    # edges
    m = len(edges)
    edgeplayers = []
    for j in range(0,m):
        edgeplayers.append(Player('edge', f'e{j}', 1))
    # filler    
    k = len(covers)
    roundLimit1 = int(np.ceil(np.log(n-k)))
    fillerplayers = []
    for r in range(1, roundLimit1+1):
        for i in range(1,k+1):
            #for j in vertexplayers:
            fillerplayers.append(Player('filler', f'f^{r}_v#{i}', r))
    roundLimit2 = int(np.ceil(np.log(n-k)) + np.ceil(np.log(m)))
    for r in range(roundLimit1+1, roundLimit2+1):
        for i in range(1,k+1):
            #for j in edgeplayers:
            fillerplayers.append(Player('filler', f'f^{r}_e#{i}', r))
    # holder
    holderplayers = []
    for ei in edgeplayers:
        for l in range(0, 2**roundLimit1 - 1):
            holderplayers.append(Player('holder', f'h^{l}_{ei.shorthand}', 1))
            
    for fi in fillerplayers:
        fillerplayerR = fi.roundNum
        for l in range(0, 2**fillerplayerR -1):
            holderplayers.append(Player('holder', f'h^{l}_{fi.shorthand}', 1))
    kfiller = 2**(roundLimit2 + int(np.ceil(np.log(k+1)))+1) - 1
    for l in range(0, kfiller):
        holderplayers.append(Player('holder', f'h^{l}_o', 1))
    return {
        'objectivePlayer': objectiveplayer,
        'vertexPlayers': vertexplayers, 
        'edgePlayers': edgeplayers,
        'fillerPlayers': fillerplayers,
        'holderPlayers': holderplayers
    }

In [93]:
chosenGraph = graph1
nodes = getNodes(chosenGraph)
edges = getEdges(chosenGraph, False)
covers = ["b"]
players = buildPlayers(nodes, edges, covers)

print(f'Generated players: {playerNums}')

#rounds of number of players
def getFullPlayers(playerNums):
    n = 0
    stopFlag = False
    while (not stopFlag):
        if playerNums > 2**n:
            n += 1
        else:
            stopFlag = True
    return 2**n

def fillPlayers(playersList):
    playerNums = np.sum([
        len(playersList['objectivePlayer']),
        len(playersList['vertexPlayers']), 
        len(playersList['edgePlayers']),
        len(playersList['fillerPlayers']),
        len(playersList['holderPlayers'])
    ])
    if getFullPlayers(playerNums) > playerNums:
        for i in range(0, getFullPlayers(playerNums) - playerNums):
            players['holderPlayers'].append(Player('holder', f'h^{i}_ammend', 1))
    return playersList

Generated players: 30


In [94]:
players =fillPlayers(players)

In [95]:
for ptype in players.values():
    for p in ptype:
        print(p.self_introduce())

Player o of type objective for round 1
Player v0 of type vertex for round 1
Player v1 of type vertex for round 1
Player v2 of type vertex for round 1
Player v3 of type vertex for round 1
Player e0 of type edge for round 1
Player e1 of type edge for round 1
Player f^1_v#1 of type filler for round 1
Player f^2_e#1 of type filler for round 2
Player h^0_e0 of type holder for round 1
Player h^0_e1 of type holder for round 1
Player h^0_f^1_v#1 of type holder for round 1
Player h^0_f^2_e#1 of type holder for round 1
Player h^1_f^2_e#1 of type holder for round 1
Player h^2_f^2_e#1 of type holder for round 1
Player h^0_o of type holder for round 1
Player h^1_o of type holder for round 1
Player h^2_o of type holder for round 1
Player h^3_o of type holder for round 1
Player h^4_o of type holder for round 1
Player h^5_o of type holder for round 1
Player h^6_o of type holder for round 1
Player h^7_o of type holder for round 1
Player h^8_o of type holder for round 1
Player h^9_o of type holder for r

In [107]:
def matchingPhase1(playerList):
    seeds = []
    holderString = []
    for h in playerList['holderPlayers']:
        holderString.append(h.shorthand)
    vertexString = []
    for v in playerList['vertexPlayers']:
        vertexString.append(v.shorthand)
    edgeString = []
    for e in playerList['edgePlayers']:
        edgeString.append(e.shorthand)
    fillerString = []
    for f in playerList['fillerPlayers']:
        fillerString.append(f.shorthand)
    for edgePlayer in edgeString:
        for holdPlayer in holderString:
            if edgePlayer in holdPlayer:
                seeds.append((edgePlayer, holdPlayer))
    for fillerPlayer in fillerString:
        for holdPlayer in holderString:
            if fillerPlayer in holdPlayer:
                seeds.append((fillerPlayer, holdPlayer))
    return seeds

In [108]:
matchingPhase1(players)

[('e0', 'h^0_e0'),
 ('e1', 'h^0_e1'),
 ('f^1_v#1', 'h^0_f^1_v#1'),
 ('f^2_e#1', 'h^0_f^2_e#1'),
 ('f^2_e#1', 'h^1_f^2_e#1'),
 ('f^2_e#1', 'h^2_f^2_e#1')]

In [104]:
"a" in "bc"

False

In [4]:
# idea comes from this very very old paper
# https://www.barton.edu/pdf/faculty-publications/bengtson-winning-probabilities-publication.pdf

import numpy as np

p = [[1, 0.434, 0.337, 0.298, 0.262, 0.277, 0.312, 0.485],
[0.566, 1, 0.571, 0.503, 0.42, 0.456, 0.516, 0.546],
[0.663, 0.429, 1, 0.432, 0.356, 0.389, 0.463, 0.424],
[0.702, 0.497, 0.568, 1, 0.416, 0.454, 0.514, 0.492],
[0.738, 0.58, 0.644, 0.584, 1, 0.592, 0.596, 0.576],
[0.723, 0.544, 0.611, 0.546, 0.408, 1, 0.56, 0.538],
[0.688, 0.484, 0.537, 0.486, 0.404, 0.44, 1, 0.478],
[0.515, 0.454, 0.576, 0.508, 0.424, 0.462, 0.522, 1]]



# bracket kingstone is making
tournament = [[(0,1),(2,3),(4,5),(6,7)],#round 1 
             [(0,2),(4,6)],#round 2
             [(0,4)]]#round3

#gives probability of winnig i agains j (philip func)
def P(i,j):
    return p[i][j]
 
#just a function can go through braket and find the opponent
def opponent(player_i,round):
    for (i,j) in tournament[round-1]:
        if i == player_i:
            return j
        elif j == player_i:
            return i
    return None

#go to the refrence for this one
def Win_rate(player_i,round):
    if round == 0 : return 1
    opp_player = opponent(player_i,round)
    if opp_player == None: return 0
    return Win_rate(player_i,round-1) * P(player_i,opp_player) * Win_rate(opp_player,round-1)

Win_rate(2,3)

0